In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import train_test_split

### 기존 raw train.csv

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/open/데이터중급반_21조_GOAT_train.csv.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train

,Unnamed: 0,base_hour,connect_code,day_of_week,lane_count,maximum_speed_limit,multi_linked,road_rating,road_type,target,weight_restricted,month,year,day,end_turn_restricted_있음,start_turn_restricted_있음,road_name_길,road_name_로,road_name_일반국도,road_name_지방도
0,0,0.739130,0.0,3,0.0,0.6,0.0,0.75,1.0,52.0,0.648,0.454545,1.0,0.733333,0,0,0,0,0,1
1,1,0.913043,0.0,3,0.5,0.6,0.0,0.00,0.0,30.0,0.000,0.545455,1.0,0.900000,0,1,0,0,1,0
2,2,0.304348,0.0,6,0.5,1.0,0.0,0.00,0.0,61.0,0.000,0.818182,0.0,0.300000,0,0,0,0,1,0
3,3,0.565217,0.0,4,0.5,0.4,0.0,1.00,0.0,20.0,0.000,0.181818,1.0,0.333333,0,0,0,1,0,0
4,4,0.347826,0.0,1,0.5,1.0,0.0,0.00,0.0,38.0,0.000,0.818182,0.0,0.133333,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4131749,4701210,0.608696,0.0,2,0.0,0.0,0.0,1.00,0.0,25.0,0.000,0.000000,1.0,0.133333,0,0,0,1,0,0
4131750,4701211,0.608696,0.0,3,0.0,0.4,0.0,0.00,0.0,27.0,0.000,1.000000,0.0,0.966667,0,0,0,0,1,0
4131751,4701214,0.956522,0.0,0,0.5,0.6,0.0,0.00,0.0,30.0,0.000,0.454545,1.0,0.400000,0,0,0,0,1,0
4131752,4701215,0.086957,0.0,2,0.5,1.0,0.0,0.00,0.0,73.0,0.000,0.818182,0.0,0.633333,0,0,0,0,1,0


In [ ]:
train_sp = train.select_dtypes(exclude = 'object')

In [ ]:
train_sp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4701217 entries, 0 to 4701216
Data columns (total 16 columns):
 #   Column               Dtype  
---  ------               -----  
 0   base_date            int64  
 1   base_hour            int64  
 2   lane_count           int64  
 3   road_rating          int64  
 4   multi_linked         int64  
 5   connect_code         int64  
 6   maximum_speed_limit  float64
 7   vehicle_restricted   float64
 8   weight_restricted    float64
 9   height_restricted    float64
 10  road_type            int64  
 11  start_latitude       float64
 12  start_longitude      float64
 13  end_latitude         float64
 14  end_longitude        float64
 15  target               float64
dtypes: float64(9), int64(7)
memory usage: 573.9 MB


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326833 entries, 0 to 326832
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   day_of_week            326833 non-null  int64  
 1   base_hour              326833 non-null  int64  
 2   lane_count             326833 non-null  int64  
 3   road_rating            326833 non-null  int64  
 4   road_name              326833 non-null  int64  
 5   maximum_speed_limit    287489 non-null  float64
 6   weight_restricted      326833 non-null  float64
 7   road_type              326833 non-null  int64  
 8   start_node_name        326833 non-null  int64  
 9   start_latitude         326833 non-null  float64
 10  start_longitude        326833 non-null  float64
 11  start_turn_restricted  326833 non-null  int64  
 12  end_node_name          326833 non-null  int64  
 13  end_latitude           326833 non-null  float64
 14  end_longitude          326833 non-nu

In [ ]:
train

,day_of_week,base_hour,lane_count,road_rating,road_name,maximum_speed_limit,weight_restricted,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target,season
0,1,17,1,106,45,0.6,32400.0,3,344,33.427747,126.662612,0,344,33.427749,126.662335,0,52.0,1
1,1,21,2,103,33,0.6,0.0,0,26,33.500730,126.529107,1,2,33.504811,126.526240,0,30.0,0
2,4,7,2,103,35,1.0,0.0,0,380,33.279145,126.368598,0,155,33.280072,126.362147,0,61.0,3
3,0,13,2,107,56,0.4,0.0,0,43,33.246081,126.567204,0,178,33.245565,126.566228,0,20.0,1
4,6,8,2,103,34,1.0,0.0,0,232,33.462214,126.326551,0,234,33.462677,126.330152,0,38.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326828,5,20,2,103,34,NaN,0.0,0,277,33.492699,126.428241,0,277,33.492539,126.426231,0,40.0,3
326829,2,16,3,106,52,NaN,0.0,0,214,33.249949,126.505664,0,292,33.249583,126.507874,0,32.0,2
326830,5,10,1,103,35,NaN,32400.0,3,341,33.261655,126.452658,0,341,33.261690,126.453102,0,55.0,2
326831,3,10,2,103,34,NaN,0.0,0,269,33.465318,126.908612,0,202,33.470372,126.902793,0,48.0,2


In [ ]:
train.columns

Index(['Unnamed: 0', 'base_hour', 'connect_code', 'day_of_week', 'lane_count',
       'maximum_speed_limit', 'multi_linked', 'road_rating', 'road_type',
       'target', 'weight_restricted', 'month', 'year', 'day',
       'end_turn_restricted_있음', 'start_turn_restricted_있음', 'road_name_길',
       'road_name_로', 'road_name_일반국도', 'road_name_지방도'],
      dtype='object')

In [ ]:
train_x = train[['base_hour', 'connect_code', 'day_of_week', 'lane_count',
       'maximum_speed_limit', 'multi_linked', 'road_rating', 'road_type','weight_restricted', 'month', 'year', 'day',
       'end_turn_restricted_있음', 'start_turn_restricted_있음', 'road_name_길',
       'road_name_로', 'road_name_일반국도', 'road_name_지방도']]

In [ ]:
train_y = train['target']

In [ ]:
train

,day_of_week,base_hour,lane_count,road_rating,road_name,maximum_speed_limit,weight_restricted,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target,season
0,1,17,1,106,45,0.6,32400.0,3,344,33.427747,126.662612,0,344,33.427749,126.662335,0,52.0,1
1,1,21,2,103,33,0.6,0.0,0,26,33.500730,126.529107,1,2,33.504811,126.526240,0,30.0,0
2,4,7,2,103,35,1.0,0.0,0,380,33.279145,126.368598,0,155,33.280072,126.362147,0,61.0,3
3,0,13,2,107,56,0.4,0.0,0,43,33.246081,126.567204,0,178,33.245565,126.566228,0,20.0,1
4,6,8,2,103,34,1.0,0.0,0,232,33.462214,126.326551,0,234,33.462677,126.330152,0,38.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326828,5,20,2,103,34,NaN,0.0,0,277,33.492699,126.428241,0,277,33.492539,126.426231,0,40.0,3
326829,2,16,3,106,52,NaN,0.0,0,214,33.249949,126.505664,0,292,33.249583,126.507874,0,32.0,2
326830,5,10,1,103,35,NaN,32400.0,3,341,33.261655,126.452658,0,341,33.261690,126.453102,0,55.0,2
326831,3,10,2,103,34,NaN,0.0,0,269,33.465318,126.908612,0,202,33.470372,126.902793,0,48.0,2


In [ ]:
X_train , X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3, random_state=111)

In [ ]:
LR = lgb.LGBMRegressor(random_state=111).fit(X_train, y_train)

In [ ]:
pred = LR.predict(X_test)

In [ ]:
pred

array([35.3784969 , 50.95828445, 31.65184442, ..., 51.69023127,
       47.28425358, 44.76940387])

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mean_absolute_error(y_test, pred)

7.293452757232722